In [1]:
import requests
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
Задание
Скачать все вакансии СБЕРа с ХХ (у апи есть ограничение в 2000, а также captcha подумайте как их можно обойти)
Извлеките все описания этих вакансий
Создайте DataFrame добавьте поле skills
Переведите даты публикаций в datetime
Постройте график опубликованных вакансий по датам
Переведите даты в дни недели, и определите день недели, в который больше всего публикуют вакансий
Найдите те вакансии с использованием python, которые вам интересны
Определите по полю skills какие навыки больше всего востребованы для этих вакансий
Постройте график наиболее востребованных навыков для аналитика данных

In [2]:
sber = '3529'
page = 1
num_per_page = 100
moscow = 1
url = f'https://api.hh.ru/vacancies'
params = {
    'employer_id':sber,
    'page':page,
    'per_page':num_per_page,
    'area':moscow,
}
res = requests.get(url, params=params)

In [3]:
vacancies = res.json()

In [4]:
vacancies.keys()

dict_keys(['items', 'found', 'pages', 'per_page', 'page', 'clusters', 'arguments', 'alternate_url'])

In [5]:
# собираем вакансии со всех pages в один список
all_vacs = []
for i in range(vacancies.get('pages')):
    res = requests.get(url, params=params)
    vacancies = res.json()
    vacancies = vacancies.get('items')
    for i in vacancies:
        all_vacs.append(i)
    page += 1
len(all_vacs)

1200

In [6]:
# строим DataFrame
df = pd.DataFrame(all_vacs)
df

,id,premium,name,department,has_test,response_letter_required,area,salary,type,address,...,alternate_url,relations,employer,snippet,contacts,schedule,working_days,working_time_intervals,working_time_modes,accept_temporary
0,66523856,False,Аналитик SQL,"{'id': '3529-3529-it', 'name': 'Сбер. IT'}",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",None,...,https://hh.ru/vacancy/66523856,[],"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",{'requirement': 'Высшее образование (математик...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False
1,67908794,False,Стажер Sber AI Lab,"{'id': '3529-3529-it', 'name': 'Сбер. IT'}",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 55000, 'to': 57000, 'currency': 'RUR'...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': 'Кутузовский прос...",...,https://hh.ru/vacancy/67908794,[],"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",{'requirement': 'Имеешь хорошие знания математ...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False
2,67112685,False,Стрим Лид (Руководитель направления),"{'id': '3529-3529-prof', 'name': 'Сбер для экс...",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",None,...,https://hh.ru/vacancy/67112685,[],"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",{'requirement': 'Навыки постановки задач и фор...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False
3,68213260,False,Project Manager общекоммерческих проектов Цифр...,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': 'Кутузовский прос...",...,https://hh.ru/vacancy/68213260,[],"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",{'requirement': 'Имеете опыт работы менеджеров...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False
4,66223607,False,С++ разработчик,"{'id': '3529-3529-it', 'name': 'Сбер. IT'}",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",None,...,https://hh.ru/vacancy/66223607,[],"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",{'requirement': 'Хорошее знание STL. Знание C ...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,67579448,False,Руководитель направления развития ЦОД Сбера,"{'id': '3529-3529-prof', 'name': 'Сбер для экс...",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",None,...,https://hh.ru/vacancy/67579448,[],"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",{'requirement': 'Глубокое понимание индустрии ...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False
1196,67900726,False,Senior Data Analyst (в трайб «Массовая персона...,"{'id': '3529-3529-prof', 'name': 'Сбер для экс...",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",None,...,https://hh.ru/vacancy/67900726,[],"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",{'requirement': 'высшее образование по направл...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False
1197,68261867,False,Аналитик в Департамент инвестиционной деятельн...,"{'id': '3529-3529-prof', 'name': 'Сбер для экс...",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",None,...,https://hh.ru/vacancy/68261867,[],"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",{'requirement': 'Высшее финансовое или математ...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False
1198,67751425,False,Аналитик (SberWorks),"{'id': '3529-3529-it', 'name': 'Сбер. IT'}",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",None,...,https://hh.ru/vacancy/67751425,[],"{'id': '3529', 'name': 'СБЕР'

In [7]:
# приводим столбцы к читаемому формату
df['area'] = df.area.apply(lambda x: x.get('name'))
df['type'] = df.type.apply(lambda x: x.get('name'))
df['employer'] = df.employer.apply(lambda x: x.get('name'))
df['skills_1'] = df.snippet.apply(lambda x: x.get('requirement'))
df['skills_2'] = df.snippet.apply(lambda x: x.get('responsibility'))
df['schedule'] = df.schedule.apply(lambda x: x.get('name'))

In [8]:
# создаем столбец skills
df['main_skills'] = df['skills_1'] + df['skills_2']

df.rename(columns={'main_skills': 'skills'}, inplace=True)

In [11]:
# удаляем лишние колонки
df.drop(columns=['skills_1', 'skills_2', 'salary'], inplace=True)
df.columns

df.drop(columns=['address'], inplace=True)
df.drop(columns=['snippet'], inplace=True)

df.drop(columns=['department'], inplace=True)

Index(['id', 'premium', 'name', 'department', 'has_test',
       'response_letter_required', 'area', 'type', 'address', 'response_url',
       'sort_point_distance', 'published_at', 'created_at', 'archived',
       'apply_alternate_url', 'insider_interview', 'url', 'adv_response_url',
       'alternate_url', 'relations', 'employer', 'snippet', 'contacts',
       'schedule', 'working_days', 'working_time_intervals',
       'working_time_modes', 'accept_temporary', 'skills'],
      dtype='object')

In [20]:
vacs_df = df.copy()

In [27]:
vacs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype                                
---  ------                    --------------  -----                                
 0   id                        1200 non-null   object                               
 1   premium                   1200 non-null   bool                                 
 2   name                      1200 non-null   object                               
 3   has_test                  1200 non-null   bool                                 
 4   response_letter_required  1200 non-null   bool                                 
 5   area                      1200 non-null   object                               
 6   type                      1200 non-null   object                               
 7   response_url              0 non-null      object                               
 8   sort_point_distance       0 non-null  

In [26]:
# переводим дату публикации в datetime
vacs_df['published_at'] = pd.to_datetime(vacs_df.published_at)
vacs_df

,id,premium,name,has_test,response_letter_required,area,type,response_url,sort_point_distance,published_at,...,alternate_url,relations,employer,contacts,schedule,working_days,working_time_intervals,working_time_modes,accept_temporary,skills
0,66523856,False,Аналитик SQL,False,False,Москва,Открытая,None,None,2022-07-05 18:51:27+03:00,...,https://hh.ru/vacancy/66523856,[],СБЕР,None,Полный день,[],[],[],False,Высшее образование (математика/физика/программ...
1,67908794,False,Стажер Sber AI Lab,False,False,Москва,Открытая,None,None,2022-07-19 11:32:28+03:00,...,https://hh.ru/vacancy/67908794,[],СБЕР,None,Полный день,[],[],[],False,Имеешь хорошие знания математики и алгоритмов....
2,67112685,False,Стрим Лид (Руководитель направления),False,False,Москва,Открытая,None,None,2022-07-19 10:17:56+03:00,...,https://hh.ru/vacancy/67112685,[],СБЕР,None,Полный день,[],[],[],False,Навыки постановки задач и формулирования требо...
3,68213260,False,Project Manager общекоммерческих проектов Цифр...,False,False,Москва,Открытая,None,None,2022-07-27 16:25:07+03:00,...,https://hh.ru/vacancy/68213260,[],СБЕР,None,Полный день,[],[],[],False,"Имеете опыт работы менеджеров проектов от 1,5 ..."
4,66223607,False,С++ разработчик,False,False,Москва,Открытая,None,None,2022-07-21 17:52:19+03:00,...,https://hh.ru/vacancy/66223607,[],СБЕР,None,Полный день,[],[],[],False,Хорошее знание STL. Знание C и C++11/14/17. Ра...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,67579448,False,Руководитель направления развития ЦОД Сбера,False,False,Москва,Открытая,None,None,2022-07-07 14:33:11+03:00,...,https://hh.ru/vacancy/67579448,[],СБЕР,None,Полный день,[],[],[],False,Глубокое понимание индустрии ЦОДов. Опыт работ...
1196,67900726,False,Senior Data Analyst (в трайб «Массовая персона...,False,False,Москва,Открытая,None,None,2022-07-19 09:43:05+03:00,...,https://hh.ru/vacancy/67900726,[],СБЕР,None,Полный день,[],[],[],False,высшее образование по направлению: прикладная ...
1197,68261867,False,Аналитик в Департамент инвестиционной деятельн...,False,False,Москва,Открытая,None,None,2022-07-28 17:38:25+03:00,...,https://hh.ru/vacancy/68261867,[],СБЕР,None,Полный день,[],[],[],False,Высшее финансовое или математическое образован...
1198,67751425,False,Аналитик (SberWorks),False,False,Москва,Открытая,None,None,2022-07-13 17:35:49+03:00,...,https://hh.ru/vacancy/67751425,[],СБЕР,None,Полный день,[],[],[],False,Опыт работы в роле бизнес или системного анали...


In [31]:
# создаем столбец с днями недели
vacs_df['day_of_week'] = vacs_df.published_at.dt.dayofweek
vacs_df['day_of_week'] = vacs_df.day_of_week.apply(lambda x: x + 1)

weekdays = {
    1: 'Понедельник',
    2: 'Вторник',
    3: 'Среда',
    4: 'Четверг',
    5: 'Пятница',
    6: 'Суббота',
    7: 'Воскресенье'
}

vacs_df['day_of_week'] = [weekdays.get(i) for i in vacs_df.day_of_week]

In [50]:
# находим, в какой день недели опубликовано больше всего вакансий
max_vacs_weekday = vacs_df.groupby(['day_of_week']).size()
max_vacs_weekday = max_vacs_weekday[max_vacs_weekday == max_vacs_weekday.max()].index[0]

print(f'День в который больше всего публикуют вакансий: {max_vacs_weekday}')

День в который больше всего публикуют вакансий: Среда


In [52]:
vacs_df.head()

,id,premium,name,has_test,response_letter_required,area,type,response_url,sort_point_distance,published_at,...,relations,employer,contacts,schedule,working_days,working_time_intervals,working_time_modes,accept_temporary,skills,day_of_week
0,66523856,False,Аналитик SQL,False,False,Москва,Открытая,None,None,2022-07-05 18:51:27+03:00,...,[],СБЕР,None,Полный день,[],[],[],False,Высшее образование (математика/физика/программ...,Вторник
1,67908794,False,Стажер Sber AI Lab,False,False,Москва,Открытая,None,None,2022-07-19 11:32:28+03:00,...,[],СБЕР,None,Полный день,[],[],[],False,Имеешь хорошие знания математики и алгоритмов....,Вторник
2,67112685,False,Стрим Лид (Руководитель направления),False,False,Москва,Открытая,None,None,2022-07-19 10:17:56+03:00,...,[],СБЕР,None,Полный день,[],[],[],False,Навыки постановки задач и формулирования требо...,Вторник
3,68213260,False,Project Manager общекоммерческих проектов Цифр...,False,False,Москва,Открытая,None,None,2022-07-27 16:25:07+03:00,...,[],СБЕР,None,Полный день,[],[],[],False,"Имеете опыт работы менеджеров проектов от 1,5 ...",Среда
4,66223607,False,С++ разработчик,False,False,Москва,Открытая,None,None,2022-07-21 17:52:19+03:00,...,[],СБЕР,None,Полный день,[],[],[],False,Хорошее знание STL. Знание C и C++11/14/17. Ра...,Четверг


In [59]:
vacs_df[vacs_df['skills'] == ''] = None

In [66]:
# находим интересные вакансии
interesting_vacs = vacs_df[vacs_df.skills.str.contains('python', na=False, case=False) 
        & vacs_df.name.str.contains('Разработчик', na=False, case=False)]['name'].unique()
interesting_vacs

array(['Разработчик Python (риск-стратегий)', 'Backend разработчик'],
      dtype=object)

In [77]:
# находим востребованные навыки
vacs_df[vacs_df.name.isin(interesting_vacs)]['skills'].unique()

array(['Высшее образование. Знание Python: написание, отладка и тестирование кода. Опыт в написании сервисов на Python. Опыт в анализе логов сервиса...Внедрение новых и сопровождение (доработка/внесение изменений) текущих моделей оценки кредитного риска в автоматизированных системах Банка. Подготовка технической документации для...',
       'Отличное знание одного из ООП языков: Java, C#, Python, PHP. Знание основного фреймворка языка: SpringBoot, .NET, Django, Symfony. Разбор задачи, описанной в терминах бизнеса. Работа по фреймворку Scram. Чтение и внесение доработок в чужом коде. Проведение code-review. '],
      dtype=object)